![png](../../../imagenes/logotipo-axity-ppt.png)

# Filtrado Basado en Contenido

Los sistemas de recomendación son una colección de algoritmos utilizados para recomendar artículos a los usuarios, basándose en información que proviene del usuario. Estos sistemas se han vuelto omnipresentes y pueden encontrarse en tiendas en línea, dases de datos de películas y buscadores de trabajos.  

En este ejercicio, exploraremos los sistemas de recomendación basados el Filtrado Basado en Contenido.

## Los datos

Utilizaremos el conjunto de datos [Movie Lens de Kaggle](https://www.kaggle.com/shubhammehta21/movie-lens-small-latest-dataset).  

Este contiene las calificaciones de MovieLens, un servicio de recomendación de películas.  

El conjunto contiene 100,000 calificaciones de 1000 usuarios para 1700 películas.

Contiene las siguientes columnas:

**movies.csv**
* movieId: identificador de la película
* title: título de la película
* genres: géneros de la película, separado por |

**ratings.csv**
* userId: Identificador del usuario
* movieId: Identificador de la película
* rating: Calificación (0 - 5)

## Carga de datos y preprocesamiento

In [1]:
# Importemos las librerías
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [2]:
# Carguemos los datos
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [3]:
# Revisemos los datos
movies_df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

Cada pelicula tiene un ID único, un título con el año en que fue liberada y varios géneros en el mismo campo.  
Removamos el año del título y coloquémoslo en una columna adicional.

In [4]:
# Usemos expresiones regulares para encontrar el año almacenado entre paréntesis
# Especificamos los paréntesis, para no tener conflictos con las películas que tienen años en sus títulos
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

# Removamos los paréntesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

# Removamos el año del título
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

# Apliquemos la función strip, para quitar los espacios en blanco al final
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [5]:
# Veamos los resultados
movies_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

Ahora separemos los valores en la columna de géneros en una lista de géneros, para utilizarla posteriormente. Esto puede realizarse mediante la función split de Python.

In [6]:
# Cada género está separado por un | asi que solamente tenemos que llamar la función split con |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                     [Adventure, Children, Fantasy]  1995  
2                                  [Comedy, Romance]  1995  
3                           [Comedy, Drama, Romance]  1995  
4                                           [Comedy]  1995

Utilizaremos one hot encoding para separar la lista de géneros, para que los podamos utilizar como características.  
Almacenaremos estos géneros en un dataframe adicional, dado que no lo utilizaremos en nuestro primer sistema de recomendación.  

In [7]:
# Copiemos el dataframe en uno nuevo, dado que no utilizaremos la información para nuestro primer sistema de recomendación
moviesWithGenres_df = movies_df.copy()

# Para cada renglón en el dataframe, iteremos por la lista de géneros y coloquemos un 1 en la columna correspondiente
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

# Sustituyamos los valores NaN por 0, para indicar que la película no aplica para el género de esa columna
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  Adventure  \
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                     [Adventure, Children, Fantasy]  1995        1.0   
2                                  [Comedy, Romance]  1995        0.0   
3                           [Comedy, Drama, Romance]  1995        0.0   
4                                           [Comedy]  1995        0.0   

   Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0        1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1        0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
2        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
3        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
4        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

   Sci-Fi  War  Musical  Documentary  IMAX  Western  Film-Noir  \
0     0.0  0.0      0.0          0.0   0.0      0.0        0.0   
1     0.0  0.0      0.0          0.0   0.0      0.0        0.0   
2     0.0  0.0      0.0          0.0   0.0      0.0        0.0   
3     0.0  0.0      0.0          0.0   0.0      0.0        0.0   
4     0.0  0.0      0.0          0.0   0.0      0.0        0.0   

   (no genres listed)  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  

[5 rows x 24 columns]

Ahora revisemos el dataframe de calificaciones

In [8]:
ratings_df.head()

userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Cada renglón en el dataframe tiene un ID de usuario, asociado con la menos una película, una calificación y una marca de tiempo que indica cuando se calificó. No necesitaremos esta última columna, asi que la eliminaremos.

In [9]:
# Eliminemos la columna de la marca de tiempo
ratings_df = ratings_df.drop('timestamp', 1)

Revisemos la versión final del dataframe

In [10]:
ratings_df.head()

userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0

## Filtrado Basado en Contenido

Veamos como implementar el algoritmo **Basado en Contenido (Item-Item)**. Esta técnica trata de identificar los aspectos favoritos de un artículo para un usuario y recomienda artículos que cuentan con esos aspectos. En nuestro caso, trataremos de identificar los géneros favoritos del usuario activo, a partir de las películas y las calificaciones que ha proporcionado.  

Iniciemos con la creación de un usuario activo ficticio, al que le recomendaremos las películas.  

Nota: para agregar mas películas, solo incrementa el número de elementos en la variable *userInput*. Asegúrate de escribir los títulos con la primera letra en mayúscula y si la película inicia con "The", como "The Matrix", escribela de la siguiente manera: 'Matrix, The' .  

In [11]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

title  rating
0  Breakfast Club, The     5.0
1            Toy Story     3.5
2              Jumanji     2.0
3         Pulp Fiction     5.0
4                Akira     4.5

#### Agregar el ID de la película al usuario activo
Una vez que tenemos completo el usuario activo, extraigamos el ID de la película del dataframe de películas y agreguémoslo.  

Podemos lograr esto, filtrando los renglones que contengan el título de nuestras películas e integrándolos en nuestro dataframe. También eliminaremos las columnas que sean innecesarias.   

In [12]:
# Filtremos las películas por título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

# Integrémoslas a nuestro dataframe, para obtener el ID de la película. Se realiza la integración por el título
inputMovies = pd.merge(inputId, inputMovies)

# Eliminemos la información que no utiizaremos
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)

# Dataframe final
# Nota: si agregaste una película al dataframke y no aparece, es probable que no esté en la lista de películas o esté escrita de manera diferente
inputMovies

movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0

Iniciemos con el aprendizaje de las preferencias del usuario. Obtengamos los géneros de las películas que el usuario ha visto, a partir del dataframe que contiene los géneros definidos por valores binarios (es decir, el que obtuvimos al aplicar one hot encoding).

In [14]:
# Incorporemos los géneros a las películas del usuario activo
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
257       296         Pulp Fiction   
973      1274                Akira   
1445     1968  Breakfast Club, The   

                                                 genres  year  Adventure  \
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                        [Adventure, Children, Fantasy]  1995        1.0   
257                    [Comedy, Crime, Drama, Thriller]  1994        0.0   
973              [Action, Adventure, Animation, Sci-Fi]  1988        1.0   
1445                                    [Comedy, Drama]  1985        0.0   

      Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0           1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1           0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
257         0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   
973         1.0       0.0     0.0      0.0      0.0  ...     0.0      0.0   
1445        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

      Sci-Fi  War  Musical  Documentary  IMAX  Western  Film-Noir  \
0        0.0  0.0      0.0          0.0   0.0      0.0        0.0   
1        0.0  0.0      0.0          0.0   0.0      0.0        0.0   
257      0.0  0.0      0.0          0.0   0.0      0.0        0.0   
973      1.0  0.0      0.0          0.0   0.0      0.0        0.0   
1445     0.0  0.0      0.0          0.0   0.0      0.0        0.0   

      (no genres listed)  
0                    0.0  
1                    0.0  
257                  0.0  
973                  0.0  
1445                 0.0  

[5 rows x 24 columns]

Solo necesitaremos los géneros separados por columnas, por lo que limpiaremos el dataframe y resetearemos los pindices, eliminando el ID de la película, el título, la columna original de géneros y el año.

In [15]:
# Reseteando los índices
userMovies = userMovies.reset_index(drop=True)

# Eliminando las columnas innecesarias
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  Action  \
0        1.0        1.0       1.0     1.0      1.0      0.0    0.0     0.0   
1        1.0        0.0       1.0     0.0      1.0      0.0    0.0     0.0   
2        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   
3        1.0        1.0       0.0     0.0      0.0      0.0    0.0     1.0   
4        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   

   Crime  Thriller  Horror  Mystery  Sci-Fi  War  Musical  Documentary  IMAX  \
0    0.0       0.0     0.0      0.0     0.0  0.0      0.0          0.0   0.0   
1    0.0       0.0     0.0      0.0     0.0  0.0      0.0          0.0   0.0   
2    1.0       1.0     0.0      0.0     0.0  0.0      0.0          0.0   0.0   
3    0.0       0.0     0.0      0.0     1.0  0.0      0.0          0.0   0.0   
4    0.0       0.0     0.0      0.0     0.0  0.0      0.0          0.0   0.0   

   Western  Film-Noir  (no genres listed)  
0      0.0        0.0                 0.0  
1      0.0        0.0                 0.0  
2      0.0        0.0                 0.0  
3      0.0        0.0                 0.0  
4      0.0        0.0                 0.0

Ahora estamos listos para aprender las preferencias del usuario activo.  

Convertiremos cada género en un peso. Podemos realizar esto utilizando las calificaciones que proporcionó el usuario activo y multiplicándolas por la tabla de géneros. Finalmente sumaremos los resultados por columna.  

En términos matemáticos, este es un porducto punto, entre una matriz y un vector, asi que podemos utilizar la función "dot" de pandas.  

In [16]:
# Revisemos las calificaciones del usuario para cada película
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [17]:
# Realicemos el producto punto
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])

# Revisemos el resultado
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Ahora tenemos los pesos por cada preferencia del usuario activo o el perfil del usuario. Mediante este, podemos recomendar las películas que satisfagan las preferencias del usuario.  

Realicemos la extracción de la tabla de géneros del dataframe original.

In [18]:
# Obtengamos los géneros de cada película de nuestro dataframe original
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

# Eliminemos la información que no es necesaria
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  \
movieId                                                                    
1              1.0        1.0       1.0     1.0      1.0      0.0    0.0   
2              1.0        0.0       1.0     0.0      1.0      0.0    0.0   
3              0.0        0.0       0.0     1.0      0.0      1.0    0.0   
4              0.0        0.0       0.0     1.0      0.0      1.0    1.0   
5              0.0        0.0       0.0     1.0      0.0      0.0    0.0   

         Action  Crime  Thriller  Horror  Mystery  Sci-Fi  War  Musical  \
movieId                                                                   
1           0.0    0.0       0.0     0.0      0.0     0.0  0.0      0.0   
2           0.0    0.0       0.0     0.0      0.0     0.0  0.0      0.0   
3           0.0    0.0       0.0     0.0      0.0     0.0  0.0      0.0   
4           0.0    0.0       0.0     0.0      0.0     0.0  0.0      0.0   
5           0.0    0.0       0.0     0.0      0.0     0.0  0.0      0.0   

         Documentary  IMAX  Western  Film-Noir  (no genres listed)  
movieId                                                             
1                0.0   0.0      0.0        0.0                 0.0  
2                0.0   0.0      0.0        0.0                 0.0  
3                0.0   0.0      0.0        0.0                 0.0  
4                0.0   0.0      0.0        0.0                 0.0  
5                0.0   0.0      0.0        0.0                 0.0

In [19]:
genreTable.shape

(9742, 20)

Con el perfil de usuario y la lista completa de películas y sus géneros, obtengamos el promedio ponderado de cada película, basándonos en el perfil del usuario y recomendemos las primeras 20 películas.  

In [20]:
# Multipliquemos los géneros por los pesos y obtengamos el promedio ponderado
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [21]:
# Ordenemos las recomendaciones en orden descendente
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

# Revisemos algunos valores
recommendationTable_df.head()

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
dtype: float64

Con esto ya tenemos nuestra tabla de recomendación

In [23]:
# Generemos la tabla final de recomendaciones
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

movieId                                              title  \
559       673                                          Space Jam   
1390     1907                                              Mulan   
2250     2987                           Who Framed Roger Rabbit?   
3460     4719                                      Osmosis Jones   
4631     6902                                      Interstate 60   
5490    26340  Twelve Tasks of Asterix, The (Les douze travau...   
5819    32031                                             Robots   
6448    51939                TMNT (Teenage Mutant Ninja Turtles)   
6455    52287                                 Meet the Robinsons   
6462    52462  Aqua Teen Hunger Force Colon Movie Film for Th...   
6879    62956                            Futurama: Bender's Game   
7441    81132                                             Rubber   
7550    85261                                    Mars Needs Moms   
8349   108540           Ernest & Célestine (Ernest et Célestine)   
8357   108932                                     The Lego Movie   
8597   117646                     Dragonheart 2: A New Beginning   
8806   130520                                               Home   
8900   134853                                         Inside Out   
9169   148775                Wizards of Waverly Place: The Movie   
9282   157865                                    Ratchet & Clank   

                                                 genres  year  
559   [Adventure, Animation, Children, Comedy, Fanta...  1996  
1390  [Adventure, Animation, Children, Comedy, Drama...  1998  
2250  [Adventure, Animation, Children, Comedy, Crime...  1988  
3460  [Action, Animation, Comedy, Crime, Drama, Roma...  2001  
4631  [Adventure, Comedy, Drama, Fantasy, Mystery, S...  2002  
5490  [Action, Adventure, Animation, Children, Comed...  1976  
5819  [Adventure, Animation, Children, Comedy, Fanta...  2005  
6448  [Action, Adventure, Animation, Children, Comed...  2007  
6455  [Action, Adventure, Animation, Children, Comed...  2007  
6462  [Action, Adventure, Animation, Comedy, Fantasy...  2007  
6879  [Action, Adventure, Animation, Comedy, Fantasy...  2008  
7441  [Action, Adventure, Comedy, Crime, Drama, Film...  2010  
7550  [Action, Adventure, Animation, Children, Comed...  2011  
8349  [Adventure, Animation, Children, Comedy, Drama...  2012  
8357  [Action, Adventure, Animation, Children, Comed...  2014  
8597  [Action, Adventure, Comedy, Drama, Fantasy, Th...  2000  
8806  [Adventure, Animation, Children, Comedy, Fanta...  2015  
8900  [Adventure, Animation, Children, Comedy, Drama...  2015  
9169  [Adventure, Children, Comedy, Drama, Fantasy, ...  2009  
9282  [Action, Adventure, Animation, Children, Comed...  2016

### Ventajas y Desventajas

##### Ventajas
* Aprende las preferencias del usuario
* Altamente personalizado para el usuario

##### Desventajas
* No toma en cuenta lo que otras personas piensan de la película, por lo que se podrían recomendar películas con calificaciones bajas
* La extracción de datos no es siempre intuitiva
* La determinación de que características le gustan o no al usuario no es siempre obvio